In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import IPython
import math
import matplotlib.pyplot as plt
import joblib
import numpy as np
import numpy.matlib
import pandas as pd
import pickle
import pydot
import pylab
import random
import scipy 
import scipy.cluster.hierarchy as shc
import scipy.stats as stats
import sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
import tabulate

from numpy import linalg
from pylab import *
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.integrate import *
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform 
from scipy.stats import *
from scipy.stats import binom, beta, expon, mvn, randint as sp_randint, shapiro, ttest_ind, bernoulli
from sklearn import metrics
from sklearn.calibration import calibration_curve
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier, VotingClassifier, BaggingRegressor, RandomForestRegressor, AdaBoostRegressor,GradientBoostingRegressor, VotingRegressor
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE, RFECV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, SGDClassifier
from sklearn.metrics import classification_report,confusion_matrix, r2_score, make_scorer, mean_squared_error, mean_absolute_error,roc_curve,accuracy_score,roc_auc_score,brier_score_loss, precision_score, recall_score,f1_score, log_loss
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import  KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.svm import SVC, LinearSVC, SVR, LinearSVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen

pd.set_option('display.max_rows',10)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
from google.colab import files
uploaded = files.upload()

Saving dcr.csv to dcr.csv



[Deep Credit Risk](http://www.deepcreditrisk.com/data--code-private.html


The data set is in panel form and reports origination and performance observations for 50,000 residential U.S. mortgage borrowers over 60 periods. For training pruposes we work with a 10% sample (the full sample can be downloaded here). The periods have been deidentified. As in the real world, loans may originate before the start of the observation period (this is an issue where loans are transferred between banks and investors as in securitization). The loan observations may thus be censored as the loans mature or borrowers refinance. The data set is a randomized selection of mortgage-loan-level data collected from the portfolios underlying U.S. residential mortgage-backed securities (RMBS) securitization portfolios and provided by International Financial Research (www.internationalfinancialresearch.org).

id: borrower id

time: time stamp of observation

orig_time: time stamp for origination

first_time: time stamp for first observation

mat_time: time stamp for maturity

res_time: time stamp for resolution

balance_time: outstanding balance at observation time

LTV_time: loan to value ratio at observation time, in %

interest_rate_time: interest rate at observation time, in %

rate_time: risk-free rate

hpi_time: house price index at observation time, base year=100

gdp_time: GDP growth at observation time, in %

uer_time: unemployment rate at observation time, in %

REtype_CO_orig_time: real estate type condominium: 1, otherwise: 0

REtype_PU_orig_time: real estate type planned urban developments: 1, otherwise: 0

REtype_SF_orig_time: single family home: 1, otherwise: 0

investor_orig_time: investor borrower: 1, otherwise: 0

balance_orig_time: outstanding balance at origination time

FICO_orig_time: FICO score at origination time, in %

LTV_orig_time: loan to value ratio at origination time, in %

Interest_Rate_orig_time: interest rate at origination time, in %

state_orig_time: US state in which the property is located

hpi_orig_time: house price index at observation time, base year=100

default_time: default observation at observation time

payoff_time: payoff observation at observation time

status_time: default (1), payoff (2) and non-default/non-payoff (0) observation at observation time

lgd_time: LGD assuming no discounting of cash flows

recovery_res: sum of all cash flows received during resolution period


In [ ]:
 
import pandas as pd
df = pd.read_csv("dcr_full.csv")

In [ ]:
df.columns

Index(['id', 'time', 'orig_time', 'first_time', 'mat_time', 'res_time',
       'balance_time', 'LTV_time', 'interest_rate_time', 'rate_time',
       'hpi_time', 'gdp_time', 'uer_time', 'REtype_CO_orig_time',
       'REtype_PU_orig_time', 'REtype_SF_orig_time', 'investor_orig_time',
       'balance_orig_time', 'FICO_orig_time', 'LTV_orig_time',
       'Interest_Rate_orig_time', 'state_orig_time', 'hpi_orig_time',
       'default_time', 'payoff_time', 'status_time', 'lgd_time',
       'recovery_res'],
      dtype='object')

In [ ]:
import warnings; warnings.simplefilter('ignore')

# Function Versions    

def versions():
    pd.set_option('display.max_rows',30)
    if IPython.version_info==(7, 6, 1, ''): 
        iPython='7.6.1' 
    elif IPython.version_info!=(7, 6, 1, ''): 
        iPython='NaN'
    table = [

                          
                          ['Python', sys.version[:5], 'NaN'],
                          ['IPython', iPython, 'IPython'],
                          ['math', NaN, 'math'],
                          ['matplotlib.pyplot, pylab', matplotlib.__version__, 'plt'],
                          ['numpy', numpy.__version__, 'np'],
                          ['pandas', pd.__version__, 'pd'],        
                          ['pickle', pickle.format_version, 'pickle'],
                          ['random', NaN, 'random'],
                          ['scipy', scipy.__version__, 'scipy'],        
                          ['sklearn', sklearn.__version__, 'sklearn'],
                          ['statsmodels', sm.__version__, 'sm'],
                          ['tabulate', tabulate.__version__, 'tabulate'],

    ]
    table=pd.DataFrame(data=table)
    table.columns = ['Package', 'Version', 'Acronym']
    print(table)
    pd.set_option('display.max_rows',10)
    
# Function dataprep

def dataprep(data_in, depvar='default_time', splitvar='time', threshold=26):
    
    df=data_in.dropna(subset=['time', 'default_time','LTV_time', 'FICO_orig_time']).copy()
    
    # Economic features
    df.loc[:, 'annuity'] = ((df.loc[:,'interest_rate_time']/(100*4))*df.loc[:,'balance_orig_time'])/(1-(1+df.loc[:,'interest_rate_time']/(100*4))**(-(df.loc[:,'mat_time']-df.loc[:,'orig_time'])))
    df.loc[:,'balance_scheduled_time']  = df.loc[:,'balance_orig_time']*(1+df.loc[:,'interest_rate_time']/(100*4))**(df.loc[:,'time']-df.loc[:,'orig_time'])-df.loc[:,'annuity']*((1+df.loc[:,'interest_rate_time']/(100*4))**(df.loc[:,'time']-df.loc[:,'orig_time'])-1)/(df.loc[:,'interest_rate_time']/(100*4))
    df.loc[:,'property_orig_time'] = df.loc[:,'balance_orig_time']/(df.loc[:,'LTV_orig_time']/100)
    df.loc[:,'cep_time']= (df.loc[:,'balance_scheduled_time'] - df.loc[:,'balance_time'])/df.loc[:,'property_orig_time']

    df.loc[:,'equity_time'] = 1-(df.loc[:,'LTV_time']/100)

    df=df.dropna(subset=['time', 'cep_time', 'equity_time'])
    
    df.loc[:,'age'] = (df.loc[:,'time']-df.loc[:,'first_time']+1)
    df.loc[df['age'] >= 40, 'age'] = 40    
    df.loc[:,'age_1'] = df.loc[:,'time']-df.loc[:,'first_time']
    df.loc[df['age_1'] >= 39, 'age_1'] = 39
    df.loc[:,'age_1f'] = df.loc[:,'age_1']
    df.loc[df['age_1f'] <= 1, 'age_1f'] = 1
    df.loc[:,'age2'] = df.loc[:,'age']**2
    
    df['vintage'] = df.loc[:,'orig_time']
    df.loc[df['vintage'] < 0, 'vintage'] = 0
    df.loc[df['vintage'] >= 30, 'vintage'] = 30
    df.loc[:,'vintage2'] = df.loc[:,'vintage']**2
    
    df.loc[:,'state_orig_time'] = pd.Categorical(df.state_orig_time, ordered=False)

    if depvar=='default_time':
        df2 = df

        df2 = df2.loc[df2['state_orig_time'] != 'AL',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'AK',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'AR',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'ND',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'SD',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'MT',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'DE',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'WV',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'VT',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'ME',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'NE',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'NH',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'MS',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'VI',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'DC',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'PR',:].copy()
        df2 = df2.loc[df2['state_orig_time'] != 'nan',:].copy() 
        
        # Splitting
        data_train = df2.loc[df2[splitvar] < threshold+1,:].copy()
        data_test = df2.loc[df2[splitvar] > threshold,:].copy()

        # PCA
        defaultrates_states_train = data_train.groupby(['time', 'state_orig_time'])['default_time'].mean().unstack(level=1).add_prefix('defaultrate_').fillna(0).reset_index(drop=False)
        defaultrates_states = df2.groupby(['time', 'state_orig_time'])['default_time'].mean().unstack(level=1).add_prefix('defaultrate_').fillna(0).reset_index(drop=False)
        
        scaler = StandardScaler().fit(defaultrates_states_train)
        defaultrates_states_train1 = scaler.transform(defaultrates_states_train)
        defaultrates_states1 = scaler.transform(defaultrates_states)

        pca = PCA()
        pca.fit(defaultrates_states_train1)  
        z_train = pca.transform(defaultrates_states_train1)
        z = pca.transform(defaultrates_states1)
        z_train = z_train[:,0:5]
        z = z[:,0:5]

        Z_train = pd.DataFrame(data=z_train, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5'])
        Z = pd.DataFrame(data=z, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5'])

        Z_train_1 = Z_train.shift(1).add_suffix('_1')
        Z_1 = Z.shift(1).add_suffix('_1')

        defaultrates_states_train2 = pd.concat([defaultrates_states_train['time'], Z_train_1], axis=1).dropna(subset=['PCA1_1']).copy()
        defaultrates_states2 = pd.concat([defaultrates_states['time'], Z_1], axis=1).dropna(subset=['PCA1_1']).copy() 

        data_train = pd.merge(data_train, defaultrates_states_train2, on='time')
        df3 = pd.merge(df2, defaultrates_states2, on='time')

        data_test = df3.loc[df3[splitvar] > threshold,:].copy()
        
        # Scaling
        X_train = data_train[['cep_time', 'equity_time', 'interest_rate_time', 'FICO_orig_time',  'gdp_time', 'PCA1_1','PCA2_1', 'PCA3_1','PCA4_1','PCA5_1']].dropna()
        X_test = data_test[['cep_time', 'equity_time', 'interest_rate_time', 'FICO_orig_time',  'gdp_time',  'PCA1_1','PCA2_1', 'PCA3_1','PCA4_1','PCA5_1']].dropna()
        
        
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        y_train = data_train['default_time'].values.reshape(-1,)
        y_test = data_test['default_time'].values.reshape(-1,)

        # Clustering
        n_clusters = 2
        kmeans = KMeans(n_clusters=n_clusters, random_state=2, verbose=0)
        kmeans.fit(X_train_scaled)

        clusters_train =kmeans.predict(X_train_scaled)
        clusters_test = kmeans.predict(X_test_scaled)

        dummies_train = pd.get_dummies(clusters_train, drop_first=True, prefix='cluster')
        dummies_test = pd.get_dummies(clusters_test, drop_first=True, prefix='cluster')
        
        X_train_scaled = np.append(X_train_scaled, dummies_train, axis=1)
        X_test_scaled = np.append(X_test_scaled, dummies_test, axis=1)

        dummies = pd.concat([dummies_train, dummies_test], axis=0, ignore_index=True)
        dummies = dummies.reindex(data_train.index)

        df3 = pd.concat([df3, dummies], axis=1).dropna(subset=['id'])
        data_train = pd.concat([data_train, dummies_train], axis=1)
        dummies_test = dummies_test.reindex(data_test.index)
        data_test  = pd.concat([data_test,  dummies_test],  axis=1)
        
    if depvar=='lgd_time':
        
        # LGD dataprep
        df2 = df.query('default_time == 1').copy()    
        df3 = resolutionbias(df2,'lgd_time','res_time','time')
        
        df3.loc[df3['lgd_time'] <= 0, 'lgd_time'] = 0.0001
        df3.loc[df3['lgd_time'] >= 1, 'lgd_time'] = 0.9999

        # Splitting
        data_train =df3.loc[df3[splitvar] < threshold+1,:].copy()
        data_test =df3.loc[df3[splitvar] > threshold,:].copy()
        
        X_train = data_train[['cep_time', 'equity_time', 'interest_rate_time', 'FICO_orig_time', 'REtype_CO_orig_time', 'REtype_PU_orig_time', 'gdp_time']]
        X_test = data_test[['cep_time', 'equity_time', 'interest_rate_time', 'FICO_orig_time', 'REtype_CO_orig_time', 'REtype_PU_orig_time', 'gdp_time']]
    
        y_train = data_train['lgd_time'].values.reshape(-1,)
        y_test = data_test['lgd_time'].values.reshape(-1,)
        
        # Scaling
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        dummies_train = pd.get_dummies(data_train.state_orig_time, drop_first=True, prefix='state_orig_time')
        dummies_test = pd.get_dummies(data_test.state_orig_time, drop_first=True, prefix='state_orig_time')

        X_train_scaled = np.append(X_train_scaled, dummies_train, axis=1)
        X_test_scaled = np.append(X_test_scaled, dummies_test, axis=1)     
    
    return df3, data_train, data_test, X_train_scaled, X_test_scaled, y_train, y_test

# Function woe

def woe(data_in, target, variable, bins, binning):
    
    df = data_in
    df2 = data_in[[target, variable]].rename(columns={target: 'Target', variable: 'Variable'}).dropna()
    
    if binning == 'True':
       df2['key'] = pd.qcut(df2.Variable, bins, labels=False, duplicates='drop')
    if binning == 'False':
       df2['key'] = df2.Variable
    table = pd.crosstab(df2.key, df2.Target, margins= True)
    table = table.drop(['All'], axis=0)
    table = table.rename(columns={1: 'deft', 0: 'nondeft'}).reset_index(drop=False)

    table.loc[:, 'fracdeft'] = table.deft/np.sum(table.deft)
    table.loc[:, 'fracnondeft'] = table.nondeft/np.sum(table.nondeft)

    table.loc[:, 'WOE'] = np.log(table.fracdeft/table.fracnondeft)
    table.loc[:, 'IV'] = (table.fracdeft-table.fracnondeft)*table.WOE
    
    table.rename(columns={'WOE': variable}, inplace=True)
    table=table.add_suffix('_WOE')
    table.rename(columns={table.columns[0]: 'key' }, inplace = True)
    WOE = table.iloc[:, [0,-2]]
    
    df = pd.merge(df, df2.key, right_index=True, left_index=True)
      
    outputWOE = pd.merge(df, WOE, on='key').drop(['key'], axis=1)
    outputIV = pd.DataFrame(data={'name': [variable], 'IV': table.IV_WOE.sum()})
    
    return outputWOE, outputIV


In [ ]:
df3, data_train, data_test, X_train_scaled, X_test_scaled, y_train, y_test = dataprep(df)